In [1]:
# Tensorflow 2.3でメモリを指定及び節約して使うためのおまじない。
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[4], 'GPU')
tf.config.experimental.set_memory_growth(physical_devices[4], True)

In [2]:
import os
dirpath = "/workdir/taki_lab/tiny-imagenet-200/"
train_dir = os.path.join(dirpath, "train")
val_dir = os.path.join(dirpath, "val")
test_dir = os.path.join(dirpath, "test")

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet_v2 import preprocess_input

N_TRAIN = 100000
N_VAL = 10000
INPUT_SIZE = (139, 139)
BATCH_SIZE = 64

train_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                              rotation_range=10,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              zoom_range=0.1,
                              fill_mode="nearest")

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_gen.flow_from_directory(train_dir,
                                               target_size=INPUT_SIZE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

val_generator = val_gen.flow_from_directory(val_dir,
                                           target_size=INPUT_SIZE,
                                           batch_size=BATCH_SIZE,
                                           class_mode="categorical")

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [ ]:
from keras import layers, Model

class 

In [4]:
from keras.models import Model
from keras import layers
from keras import Input
from keras.applications import ResNet50
from keras import backend as K
K.clear_session()


resnet50_base = ResNet50(weights="imagenet", include_top=False,
                             input_tensor=Input(shape=(139,139,3)))

x = resnet50_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation="relu")(x)
output_tensor = layers.Dense(200, activation="softmax")(x)

model = Model(inputs=resnet50_base.input, outputs=output_tensor, name="resnet50_finetuning2")

trainable_filter = int(len(resnet50_base.layers))
for layer in resnet50_base.layers[:trainable_filter]:
    layer.trainable = False
    
    if layer.name.startswith("batch_normalization"):
        layer.trainable = True

for layer in resnet50_base.layers[trainable_filter:]:
    layer.trainable = True

model.summary()

Model: "resnet50_finetuning2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 139, 139, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 145, 145, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 70, 70, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 70, 70, 64)   256         conv1_conv[0][0]                 
_______________________________________________________________________________

In [ ]:
from keras import optimizers
import wandb
from wandb.keras import WandbCallback
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# wandbの設定
wandb.init(project="tiny-image-net", name="resnet50_finetuning_aug")
config = wandb.config
config.learning_rate = 0.0001
config.lr_factor = 0.5
config.optimaizer = "RMSprop"
config.batch_size = BATCH_SIZE
config.input_size = INPUT_SIZE
config.rotation_range = 10
config.width_shift_range = 0.1
config.height_shift_range = 0.1
config.shear_range = 0.1
config.zoom_range = 0.1

# 最適化設定
model.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4),
             loss="categorical_crossentropy",
             metrics=["acc"])

early_stopping = EarlyStopping(monitor="val_acc", patience=5)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-5)

callback_list = [early_stopping, reduce_lr, WandbCallback()]


# 学習
history = model.fit(train_generator,
    steps_per_epoch=N_TRAIN//BATCH_SIZE ,
    epochs=100,
    validation_data=val_generator,
    validation_steps=N_VAL//BATCH_SIZE,
    callbacks=callback_list)

wandb: Wandb version 0.9.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/100
1562/1562 [==============================] - 721s 461ms/step - loss: 2.3598 - acc: 0.4613 - val_loss: 1.8267 - val_acc: 0.5608
Epoch 2/100
1562/1562 [==============================] - 728s 466ms/step - loss: 1.7076 - acc: 0.5771 - val_loss: 1.6977 - val_acc: 0.5869
Epoch 3/100
1562/1562 [==============================] - 733s 469ms/step - loss: 1.5530 - acc: 0.6097 - val_loss: 1.6451 - val_acc: 0.5982
Epoch 4/100
1562/1562 [==============================] - 740s 474ms/step - loss: 1.4495 - acc: 0.6317 - val_loss: 1.6209 - val_acc: 0.6049
Epoch 5/100
1562/1562 [==============================] - 739s 473ms/step - loss: 1.3705 - acc: 0.6472 - val_loss: 1.6101 - val_acc: 0.6075
Epoch 6/100
1562/1562 [==============================] - 743s 476ms/step - loss: 1.3071 - acc: 0.6606 - val_loss: 1.5845 - val_acc: 0.6163
Epoch 7/100
1562/1562 [==============================] - 740s 473ms/step - loss: 1.2522 - acc: 0.6714 - val_loss: 1.5834 - val_acc: 0.6157
Epoch 8/100
  67/1562 [>...